In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/4.info'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm

# = = = = = = = = = = = = = = =

directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = crawler_series['Info']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        if html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() == 'Continue Shopping':
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else [], {}
            for i, tr in enumerate(list_tr):
                list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
                list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
                list_text = [html_lib.unescape(text) for text in list_text]
                if len(list_text) > 2:
                    print('fuck')
                elif list_text[0] in dict_count:
                    dict_count[list_text[0]] += 1
                else:
                    dict_count[list_text[0]] = 1

            dict_specification, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                dict_specification[str(i)] = {}

                list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
                list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
                list_text = [html_lib.unescape(text) for text in list_text]
                if dict_count[list_text[0]] == 1:
                    for j, text in enumerate(list_text):
                        dict_specification[str(i)][str(j)] = text
                else:
                    if list_text[0] in dict_no:
                        dict_no[list_text[0]] += 1
                    else:
                        dict_no[list_text[0]] = 1
                    for j, text in enumerate(list_text):
                        if j == 0:
                            dict_specification[str(i)][str(j)] = f'{text}-{dict_no[list_text[0]]}'
                        else:
                            dict_specification[str(i)][str(j)] = text

            # = = = = = = = = = = = = = = =

            dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
            for i, tr in enumerate(list_tr):
                dict_kit[str(i)] = {}

                list_td = tr.xpath('./td')
                dict_kit[str(i)]['[Kit] Quantity'] = int(list_td[0].xpath('./text()')[0].strip())

                # = = = = = = = = = = = = = = =

                dict_kit[str(i)]['[Kit] Type'] = list_td[1].xpath('./text()')[0].strip()

                # = = = = = = = = = = = = = = =

                dict_ = json.loads(tr.xpath('./td[contains(@name, "listing_data_essential[")]/@value')[0])
                dict_kit[str(i)]['[Kit] Type Code'] = dict_['parttype']
                dict_kit[str(i)]['[Kit] Part Code'] = dict_['partkey']

                # = = = = = = = = = = = = = = =

                dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
                dict_kit[str(i)]['[Kit] Part Number'] = dict_2['partnumber']
                dict_kit[str(i)]['[Kit] Manufacturer'] = dict_2['catalogname']

                # = = = = = = = = = = = = = = =

                dict_kit[str(i)]['[Kit] Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['[Kit] Manufacturer']},{dict_kit[str(i)]['[Kit] Part Number']},{dict_kit[str(i)]['[Kit] Type']},{dict_kit[str(i)]['[Kit] Type Code']}'''

                # = = = = = = = = = = = = = = =

                list_a = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
                info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
                dict_kit[str(i)]['[Kit] Info'] = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['[Kit] Part Code']}&pt={dict_kit[str(i)]['[Kit] Type Code']}&Lennon=1'

                # = = = = = = = = = = = = = = =

                list_input = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
                list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

                dict_src, j = {}, 0
                for dict_ in list_dict:
                    list_src = dict_.get('Slots', [])
                    for src in list_src:
                        dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
                        j += 1

                dict_kit[str(i)]['[Kit] Dict_Src'] = dict_src

                # = = = = = = = = = = = = = = =

                dict_kit[str(i)]['[Kit] JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['[Kit] Manufacturer']};{dict_kit[str(i)]['[Kit] Part Number']};{dict_kit[str(i)]['[Kit] Type Code']};{dict_kit[str(i)]['[Kit] Part Code']}'''

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Group': crawler_series['Group'],
                                          'Type': crawler_series['Type'],
                                          'Manufacturer': crawler_series['Manufacturer'],
                                          'Part Number': crawler_series['Part Number'],
                                          'OEM': crawler_series['OEM'],
                                          'Picture': crawler_series['Picture'],
                                          'Url': crawler_series['Url'],
                                          'Json_Src': crawler_series['Json_Src'],
                                          'Info': crawler_series['Info'],
                                          'Json_Note 1': crawler_series['Json_Note 1'],
                                          'Json_Note 2': crawler_series['Json_Note 2'],
                                          'Choose': crawler_series['Choose'],
                                          'Json_Price': crawler_series['Json_Price'],
                                          'Json_Specification': json.dumps(dict_specification),
                                          'Json_Kit': json.dumps(dict_kit),
                                          'Type Code': crawler_series['Type Code'],
                                          'Part Code': crawler_series['Part Code'],
                                          'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC']})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-info_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/18 10:10:51] - 3. https://www.rockauto.com/en/moreinfo.php?pk=8304968&cc=0&pt=5756

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/18 10:10:52] - 4. https://www.rockauto.com/en/moreinfo.php?pk=8304972&cc=0&pt=5756

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/18 10:10:52] - 5. https://www.rockauto.com/en/moreinfo.php?pk=8304976&cc=0&pt=5756

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/18 10:10:53] - 1. https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/18 10:10:53] - 2. https://www.rockauto.com/en/moreinfo.php?pk=11158045&cc=0&pt=5756

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/18 10:10:53] - 7. https://www.rockauto.com/en/moreinfo.php?pk=9658984&cc=0&pt=5756

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/10/18 10:10:54] - 6. https://www.rockauto.com/en/moreinfo.php?pk=9658980&cc=0&pt=575

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3938.31it/s]

Done ~
